# Summary

This notebook explores the use of the Google Gemini Live API for text-to-speech (TTS) conversion.

In [ ]:
import wave

from google import genai

from palm_9000.settings import settings

client = genai.Client(api_key=settings.google_api_key.get_secret_value())
model = "gemini-live-2.5-flash-preview"

config = {"response_modalities": ["AUDIO"]}


async def main():
    async with client.aio.live.connect(model=model, config=config) as session:
        wf = wave.open("audio.wav", "wb")
        wf.setnchannels(1)
        wf.setsampwidth(2)
        wf.setframerate(24000)

        message = "Hello how are you?"
        await session.send_client_content(
            turns={"role": "user", "parts": [{"text": message}]}, turn_complete=True
        )

        async for response in session.receive():
            if response.data is not None:
                wf.writeframes(response.data)

            # Un-comment this code to print audio data info
            # if response.server_content.model_turn is not None:
            #      print(response.server_content.model_turn.parts[0].inline_data.mime_type)

        wf.close()


await main()